# Sharing JSON Among Languages

JavaScript Object Notation (JSON) is designed as a data interchange format.  Specifically, it is pobably used most commonly for RESTful web service (Representational state transfer).  While those might run in Python, there are numerous other programming languages and frameworks they might use; notably JavaScript is a prominent option.  Every widely used modern programming language has libraries supporting JSON.

For this lesson, we utilize an example Node.js server that is licensed as GPL v.3.0, and can be installed from Rob Kendal's GitHub repository at https://github.com/bpk68/api-server-starter.  That repository is accompanied by an excellent introductory article that describes the steps of creating a simple Node.js webserver.  I have modified that code only in minor ways for this lesson.  I will show two snippets of the JavaScript code used for illustration, but the focus here is on talking to the server from Python, not learning JavaScript or Node.js.

Let us start out by loading a few Python standard library modules, and one widely-used third-party package, that this lesson will utilize.

In [1]:
import json
from http import HTTPStatus
import requests
!cp node-server/data/users-start.json node-server/data/users.json

## Making REST Requests

This lesson—and *microservices* very commonly—will consist of calling a webserver with a *payload* formatted as JSON, and receiving a response, also usually formatted as JSON.  This structure allows many servers to interact in a manner similar to function calls, with both computation and state distributed among the various servers.  An older approach to this same architecture was XMLRPC, which in fact has a current but legacy Python standard library module `xmlrpc` to support it.

The server in this lesson provides a simple key/value database of users.  All users must have a name and a password, but they may also optionally have other data associated with them.  This design is obviously terrible from a security perspective, since "passwords" are transmitted and stored without encryption (as is other data), but that concern is not for this lesson.

The third-party package `requests` is recommended for HTTP clients, even in the Python standard library documentation itself.  However, the standard library package `urllib.request` has a less intuitive API, but will perform the same tasks if the third-party package is not available.  In our server, we can query the data it contains by making a GET request to the endpoint `/users`.  

A GET request does not pass any JSON body data; in principle it could pass URL parameters to communicate data, but that style is not used in this lesson.

In [2]:
# The URL of the RESTful server
url = 'http://localhost:3001/users'

# A response to the HTTP request
response = requests.get(url) 

# Show status code and load JSON body
print(response.status_code)
print(response.headers['Content-Type'])
json.loads(response.text)

200
application/json; charset=utf-8


{'1': {'name': 'Guido van Rossum',
  'password': 'unladenswallow',
  'details': {'profession': 'ex-BDFL'}},
 '2': {'name': 'Brendan Eich',
  'password': 'nontransitiveequality',
  'details': {'profession': 'Mozillan'}},
 '3': {'name': 'Ken Thompson',
  'password': 'p/q2-q4!',
  'details': {'profession': 'Unix Creator'}}}

### Unsuccessful Requests

A well behaving webserver will return a status code indicating the nature of the problem with a request. A very small support function will help us show the response details.

In [3]:
def phrase(response):
    for st in HTTPStatus:
        if st.value == response.status_code:
            return f"{st.value} {st.phrase}"

Trying a resource that simply does not exist.

In [4]:
url2 = 'http://localhost:3001/nonesuch'
response = requests.get(url2) 
print(phrase(response))
try:
    json.loads(response.text)
except Exception as err:
    print(err)

404 Not Found
Expecting value: line 1 column 1 (char 0)


At times we might see a status code that is neither 200 nor 404.  A 404 will not have any body, but other status codes are likely to have a body that is encoded as plain text or in another manner.  We can use this clue to decide whether to JSON decode the body.

In [5]:
url3 = 'http://localhost:3001/disabled'
response = requests.get(url3) 
print(phrase(response))
print(response.headers['Content-Type'])
response.text

410 Gone
text/plain; charset=utf-8


'Resource has been disabled'

## Pushing JSON

The way this server is configured, the same endpoint behaves differently if it receives a POST request rather than a GET request.  With a POST, a new record is added to the database.

In [6]:
headers = {'content-type': 'application/json'}
user = {"name": "David Mertz",  
        "password": "badpassword", 
        "details": {
            "profession": "Data Scientist", 
            "publisher": "INE"},
        "lucky_numbers": [12, 42, 55, 87]
       }

response = requests.post(url, data=json.dumps(user), headers=headers)
print(phrase(response))
response.text

200 OK


'new user id:4 added'

Let us make sure the database has the contents we hope for.

In [7]:
response = requests.get(url) 
json.loads(response.text)

{'1': {'name': 'Guido van Rossum',
  'password': 'unladenswallow',
  'details': {'profession': 'ex-BDFL'}},
 '2': {'name': 'Brendan Eich',
  'password': 'nontransitiveequality',
  'details': {'profession': 'Mozillan'}},
 '3': {'name': 'Ken Thompson',
  'password': 'p/q2-q4!',
  'details': {'profession': 'Unix Creator'}},
 '4': {'name': 'David Mertz',
  'password': 'badpassword',
  'details': {'profession': 'Data Scientist', 'publisher': 'INE'},
  'lucky_numbers': [12, 42, 55, 87]}}

The server may validate a POST request (or any request) in some manner, and return an appropriate status based on the JSON passed to it.

In [9]:
anon = {"password": "P4cC!^*8chWz8", "profession": "Hacker"}
response = requests.post(url, data=json.dumps(anon), headers=headers)
print(phrase(response))
response.text

400 Bad Request


'User property "name" is required'

# What the Server is Doing

The Node.js server has a bit of scaffolding to implement a server.  A very similar webserver could be implemented in Python or any other programming language.  While you may not be familiar with JavaScript, the below code should not be difficult to understand in outline.  This is the code that handle a POST to the `/users` route.

```javascript
// CREATE
app.post('/users', (req, res) => {      
    // validation
    if (! req.body.hasOwnProperty('name')) {
        res.status(400).send('User property "name" is required');
    }
```
```javascript
    // add the new user
    else {
        readFile(data => {
            const newUserId = Object.keys(data).length + 1;
            data[newUserId.toString()] = req.body;
            writeFile(JSON.stringify(data, null, 2), () => {
                res.status(200).send(`new user id:${newUserId} added`);
            });
```
```javascript
        },
            true);
    }
});
```

Although the data file that stores the database is itself simply JSON, the server explicitly parses it as JSON to assure the format.  Setting the header immediately before the call to `res.send()` is redundant because the server can detect the type from the JSON object; I added it to illustrate that we are able to explicitly set it.  Very similar APIs are present in Python websevers.

```json
// READ
app.get('/users', (req, res) => {
    fs.readFile(dataPath, 'utf8', (err, data) => {
        if (err) {
            throw err;
        }
```
```javascript
        // framework detects JSON, but set explicitly
        res.setHeader('Content-Type', 'application/json');
        res.send(JSON.parse(data));
    });
});
```